In [1]:
import tensorflow as tf
tf.random.set_seed(10)
import numpy as np
np.random.seed(10)
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
# number of turbines
num_turbines = 25

# import data
configs = os.listdir('./SCADA_sectors_v3')

In [3]:
def scada_data_extractor(fname):
    raw_data = np.loadtxt(fname,delimiter=',',skiprows=1)
    
    # Time
    time_data = raw_data[:,0:1]
    
    # Wind data - this will be added in later
    wd = raw_data[:,1:2] 
    
    # wind speeds at each turbine
    windspeed_data = raw_data[:,2:2+num_turbines].reshape(-1,25,1)
    
    # TI at each turbine
    ti_data = raw_data[:,2+num_turbines:2+2*num_turbines].reshape(-1,25,1)
    
    # power data at each turbine
    p_data = raw_data[:,2+2*num_turbines:2+3*num_turbines].reshape(-1,25,1)
    
    # x data at each turbine
    x_data = raw_data[:,2+3*num_turbines:2+5*num_turbines:2].reshape(-1,25,1)
    
    # y data at each turbine
    y_data = raw_data[:,2+3*num_turbines+1:2+5*num_turbines+1:2].reshape(-1,25,1)
    
    # copy over wd data 
    wd_data = np.zeros(shape=(raw_data.shape[0],25,1))
    wd_data[:,:,0] = wd[:,None,0]
    
    feature_data_ip = np.concatenate((wd_data,windspeed_data,ti_data,x_data,y_data),axis=-1)
    
    return feature_data_ip, p_data

In [4]:
amat_list= []
pos_list = []
ip_list = []
op_list = []

for config in configs:
    fname = './SCADA_sectors_v3/'+config+'/SCADA.csv'
    feature_data_ip, feature_data_op = scada_data_extractor(fname)
    
    ip_list.append(feature_data_ip)
    op_list.append(feature_data_op)
           
    amat = np.loadtxt('./SCADA_sectors_v3/'+config+'/node_interactions.csv',delimiter=',')
    pos = np.loadtxt('./SCADA_sectors_v3/'+config+'/positions.csv',delimiter=',',skiprows=1)
    
    for i in range(feature_data_ip.shape[0]):
        amat_list.append(amat.copy())
        pos_list.append(pos.copy())

In [5]:
ip_data = ip_list[0]
op_data = op_list[0]

for i in range(1,len(ip_list)):
    temp = ip_list[i]
    ip_data = np.concatenate((ip_data,temp),axis=0)
    
    temp = op_list[i]
    op_data = np.concatenate((op_data,temp),axis=0)

amat_data = np.asarray(amat_list)
pos_data = np.asarray(pos_list) # Ignoring pos_data for now but that can be add to ip_data

In [6]:
amat_data = amat_data.reshape(-1,amat_data.shape[1]*amat_data.shape[2],1)
print(amat_data.shape)

(10181, 625, 1)


In [7]:
ip_data = ip_data.astype('float32')
amat_data = amat_data.astype('float32')
op_data = op_data[:,:,0]

### Do train-valid-test split and shuffling

In [8]:
num_data_points = ip_data.shape[0]
train_points = int(0.7*num_data_points)
valid_points = int(0.1*num_data_points)

idx = np.arange(num_data_points)
np.random.shuffle(idx)

ip_data_train = ip_data[idx[:train_points]]
op_data_train = op_data[idx[:train_points]]
amat_data_train = amat_data[idx[:train_points]]

ip_data_valid = ip_data[idx[train_points:train_points+valid_points]]
op_data_valid = op_data[idx[train_points:train_points+valid_points]]
amat_data_valid = amat_data[idx[train_points:train_points+valid_points]]

ip_data_test = ip_data[idx[train_points+valid_points:]]
op_data_test = op_data[idx[train_points+valid_points:]]
amat_data_test = amat_data[idx[train_points+valid_points:]]

## Build Graph MPNN

In [9]:
class Message_Passer(tf.keras.layers.Layer):
    def __init__(self, node_dim):
        super(Message_Passer, self).__init__()
        self.node_dim = node_dim
        self.nn = tf.keras.layers.Dense(units=self.node_dim*self.node_dim, activation = tf.nn.relu)
      
    def call(self, node_j, edge_ij):
        
        # Embed the edge as a matrix
        A = self.nn(edge_ij)
        
        # Reshape so matrix mult can be done
        A = tf.reshape(A, [-1, self.node_dim, self.node_dim])
        node_j = tf.reshape(node_j, [-1, self.node_dim, 1])
        
        # Multiply edge matrix by node and shape into message list
        messages = tf.linalg.matmul(A, node_j)
        messages = tf.reshape(messages, [-1, tf.shape(edge_ij)[1], self.node_dim])

        return messages
    
class Message_Agg(tf.keras.layers.Layer):
    def __init__(self):
        super(Message_Agg, self).__init__()
    
    def call(self, messages):
        return tf.math.reduce_sum(messages, 2)
    

class Update_Func_GRU(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(Update_Func_GRU, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate(axis=1)
        self.GRU = tf.keras.layers.GRU(state_dim)
        
    def call(self, old_state, agg_messages):
    
        # Remember node dim
        n_nodes  = tf.shape(old_state)[1]
        node_dim = tf.shape(old_state)[2]
        
        # Reshape so GRU can be applied, concat so old_state and messages are in sequence
        old_state = tf.reshape(old_state, [-1, 1, tf.shape(old_state)[-1]])
        agg_messages = tf.reshape(agg_messages, [-1, 1, tf.shape(agg_messages)[-1]])
        concat = self.concat_layer([old_state, agg_messages])
        
        # Apply GRU and then reshape so it can be returned
        activation = self.GRU(concat)
        activation = tf.reshape(activation, [-1, n_nodes, node_dim])
        
        return activation
    
# Define the final output layer 
class Output_Regressor(tf.keras.layers.Layer):
    def __init__(self, n_nodes, intermediate_dim):
        super(Output_Regressor, self).__init__()
        self.concat_layer = tf.keras.layers.Concatenate()
        self.hidden_layer_1 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer_2 = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu)
        self.hidden_layer_3 = tf.keras.layers.Dense(units=1, activation=tf.nn.relu)
        self.output_layer = tf.keras.layers.Dense(units=n_nodes, activation=None)

        
    def call(self, nodes, edges):
                   
        # Remember node dims
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        # Tile and reshape to match edges
        state_i = tf.reshape(tf.tile(nodes, [1, 1, n_nodes]),[-1,n_nodes*n_nodes, node_dim ])
        state_j = tf.tile(nodes, [1, n_nodes, 1])
        
        # concat edges and nodes and apply MLP
        concat = self.concat_layer([state_i, edges, state_j])
                
        activation_1 = self.hidden_layer_1(concat)  
        activation_2 = self.hidden_layer_2(activation_1)
        activation_3 = self.hidden_layer_3(activation_2)
        
        output_value = self.output_layer(activation_3[:,:,0])
        
        return output_value

In [10]:
# Define a single message passing layer
class MP_Layer(tf.keras.layers.Layer):
    def __init__(self, state_dim):
        super(MP_Layer, self).__init__(self)
        self.message_passers  = Message_Passer(node_dim = state_dim) 
        self.message_aggs    = Message_Agg()
        self.update_functions = Update_Func_GRU(state_dim = state_dim)
        
        self.state_dim = state_dim         

    def call(self, nodes, edges, mask):
      
        n_nodes  = tf.shape(nodes)[1]
        node_dim = tf.shape(nodes)[2]
        
        state_j = tf.tile(nodes, [1, n_nodes, 1])

        messages  = self.message_passers(state_j, edges)

        # Do this to ignore messages from non-existant nodes
        masked =  tf.math.multiply(messages, mask)
        
        masked = tf.reshape(masked, [tf.shape(messages)[0], n_nodes, n_nodes, node_dim])

        agg_m = self.message_aggs(masked)
        
        updated_nodes = self.update_functions(nodes, agg_m)
        
        nodes_out = updated_nodes
        # Batch norm seems not to work. 
        #nodes_out = self.batch_norm(updated_nodes)
        
        return nodes_out

In [11]:
adj_input = tf.keras.Input(shape=(None,), name='adj_input')
nod_input = tf.keras.Input(shape=(None,), name='nod_input')
class MPNN(tf.keras.Model):
    def __init__(self, n_nodes, out_int_dim, state_dim, T):
        super(MPNN, self).__init__(self)   
        self.T = T
        self.embed = tf.keras.layers.Dense(units=state_dim, activation=tf.nn.relu)
        self.MP = MP_Layer(state_dim)     
        self.edge_regressor  = Output_Regressor(n_nodes,out_int_dim)
        #self.batch_norm = tf.keras.layers.BatchNormalization() 

        
    def call(self, inputs):
      
        nodes = inputs[0]
        edges = inputs[1]

        # Get distances, and create mask wherever 0 (i.e. non-existant nodes)
        # This also masks node self-interactions...
        # This assumes distance is last
        len_edges = tf.shape(edges)[-1]
        
        _, x = tf.split(edges, [len_edges -1, 1], 2)
        mask =  tf.where(tf.equal(x, 0), x, tf.ones_like(x))
        
        # Embed node to be of the chosen node dimension (you can also just pad)
        nodes = self.embed(nodes) 
        
        #nodes = self.batch_norm(nodes)
        # Run the T message passing steps
        for mp in range(self.T):
            nodes =  self.MP(nodes, edges, mask)
        
        # Regress the output values
        con_edges = self.edge_regressor(nodes, edges)
        
        
        return con_edges

In [12]:
def mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)

    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(nums, preds)))

    return reconstruction_error

def log_mse(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.math.log(tf.reduce_mean(tf.square(tf.subtract(nums, preds))))


    return reconstruction_error

def mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)


    reconstruction_error = tf.reduce_mean(tf.abs(tf.subtract(nums, preds)))


    return reconstruction_error

def log_mae(orig , preds):
 
    # Mask values for which no scalar coupling exists
    mask  = tf.where(tf.equal(orig, 0), orig, tf.ones_like(orig))

    nums  = tf.boolean_mask(orig,  mask)
    preds = tf.boolean_mask(preds,  mask)

    reconstruction_error = tf.math.log(tf.reduce_mean(tf.abs(tf.subtract(nums, preds))))

    return reconstruction_error

In [13]:
learning_rate = 0.001
def step_decay(epoch):
    initial_lrate = learning_rate
    drop = 0.1
    epochs_drop = 20.0
    lrate = initial_lrate * np.power(drop,  
           np.floor((epoch)/epochs_drop))
    tf.print("Learning rate: ", lrate)
    return lrate

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 15, restore_best_weights=True)

#lrate  =  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
#                              patience=5, min_lr=0.00001, verbose = 1)

opt = tf.optimizers.Adam(learning_rate=learning_rate)

In [14]:
mpnn = MPNN(n_nodes=25, out_int_dim = 32, state_dim = 32, T = 3)
mpnn.compile(opt, log_mae, metrics = [mae, log_mse])

## Call network once to build weights

In [15]:
mpnn.call([ip_data_train[:10],amat_data_train[:10]])

<tf.Tensor: shape=(10, 25), dtype=float32, numpy=
array([[ 1.0797135e-03, -1.2848546e-02,  1.6549561e-02,  2.9463133e-02,
         7.8990953e-03,  4.1191406e-03,  6.3671148e-03,  2.7426641e-02,
        -1.6473975e-02,  1.0674633e-04,  9.5993858e-03,  2.3424869e-02,
         1.5269341e-03,  1.7993692e-02, -1.8158950e-02,  2.0587347e-02,
         5.4382146e-03,  1.2803090e-02, -1.5448740e-02,  3.3330605e-03,
         1.8785350e-02,  6.1724260e-02, -4.6055573e-03, -2.0276092e-02,
        -3.5873525e-02],
       [-4.9472256e-03,  1.0687323e-02,  1.2348322e-02,  4.1257963e-04,
        -1.3013111e-02, -1.2119235e-02, -1.4458061e-02,  2.5175735e-03,
        -1.9541074e-02, -2.9170362e-02, -7.3225251e-03, -1.2437611e-02,
         2.1099653e-02,  1.5739635e-02, -1.3513623e-02,  8.7389117e-03,
         1.7870599e-03, -7.9250950e-03,  2.6071090e-02, -2.2223908e-02,
         5.3339349e-03, -3.9729621e-02,  3.6105834e-02, -2.1069009e-02,
        -1.0831242e-02],
       [ 6.6495920e-04, -1.9593090e-

In [16]:
batch_size = 128
epochs = 100

In [17]:
mpnn.fit([ip_data_train,amat_data_train], y = op_data_train, batch_size = batch_size, epochs = epochs, 
         callbacks = [lrate, stop_early], use_multiprocessing = False, initial_epoch = 0, verbose = 2, 
         validation_data = ([ip_data_valid,amat_data_valid],op_data_valid) )

Epoch 1/100
Learning rate:  0.001
56/56 - 68s - loss: -1.1600e+00 - mae: 0.3239 - log_mse: -1.9403e+00 - val_loss: -1.4346e+00 - val_mae: 0.2388 - val_log_mse: -2.4336e+00
Epoch 2/100
Learning rate:  0.001
56/56 - 54s - loss: -1.4758e+00 - mae: 0.2293 - log_mse: -2.4635e+00 - val_loss: -1.5024e+00 - val_mae: 0.2235 - val_log_mse: -2.4843e+00
Epoch 3/100
Learning rate:  0.001
56/56 - 55s - loss: -1.5444e+00 - mae: 0.2139 - log_mse: -2.5576e+00 - val_loss: -1.5560e+00 - val_mae: 0.2120 - val_log_mse: -2.5733e+00
Epoch 4/100
Learning rate:  0.001
56/56 - 54s - loss: -1.6039e+00 - mae: 0.2015 - log_mse: -2.6397e+00 - val_loss: -1.5979e+00 - val_mae: 0.2035 - val_log_mse: -2.5998e+00
Epoch 5/100
Learning rate:  0.001
56/56 - 55s - loss: -1.6378e+00 - mae: 0.1950 - log_mse: -2.6898e+00 - val_loss: -1.6371e+00 - val_mae: 0.1956 - val_log_mse: -2.6724e+00
Epoch 6/100
Learning rate:  0.001
56/56 - 56s - loss: -1.6500e+00 - mae: 0.1930 - log_mse: -2.7110e+00 - val_loss: -1.6628e+00 - val_mae: 0.

KeyboardInterrupt: 